In [1]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9\.]{3,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.32.post2" if v == "2.8.0" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.55.4
!pip install --no-deps trl==0.22.2

In [ ]:
%%capture
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "tyanfarm/llama3-8b-hotels-information-mixed-5epochs-finetuned", #model folder
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model)

In [ ]:
# model.save_pretrained_merged("tyanfarm/llama3-8b-hotel-addresses-finetuned-gguf", tokenizer, save_method = "merged_16bit",)
model.save_pretrained_gguf("tyanfarm/llama3-8b-hotel-information-mixed-finetuned-gguf", tokenizer, quantization_method="q8_0",)

In [ ]:
model.push_to_hub_gguf(
        "tyanfarm/llama3-8b-hotel-information-mixed-finetuned-gguf", # Change hf to your username!
        tokenizer,
        quantization_method = ["q8_0"],
        # quantization_method = ["q4_k_m"],
        token = "", # Get a token at https://huggingface.co/settings/tokens
    )

In [ ]:
chat_prompt = """
### Instruction
{}

### Input
{}

### Response
{}"""

inputs = tokenizer(
[
    chat_prompt.format(
        "", # instruction - leave this blank!
        "Renaissance Riverside Saigon Hotel ở đâu vậy", # input
        "", # output - leave this blank!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(
    **inputs,
    max_new_tokens=512,
    temperature=0.5,
    top_k=10,
    top_p=0.95)
decoded_output = tokenizer.batch_decode(outputs)[0]
tokenizer.batch_decode(outputs)